In [ ]:
import pandas as pd
import numpy as np

Emissions by Scenario: RCP/SSP Combination Pathways \
Committed Emissions from Existing Infastructure \
Considered Emissions (Not Yet Determined to Occur)

In [ ]:
# Scenario emissions pathways
# This data is in units of Mt, meaning million tonnes
scenario_CO2 = pd.read_csv('~/Desktop/Founders_Pledge/affectable_emissions_active_repo/Affectable-Emissions/data_files/planned_CO2-country_scenario.csv', index_col = 0).fillna(0).rename(columns = {'country.name.en':'country'})

# Committed emissions
# This data is in units of Mt (million tonnes per annum, from GEM)
electricity_commit = pd.read_csv('~/Desktop/Founders_Pledge/affectable_emissions_active_repo/Affectable-Emissions/data_files/electricity.committed.csv', index_col = 0).fillna(0)
# This data was in units of Gt, and gets converted to Mt (is multiplied by 1000) in the commit_Tong-GEM.ipynb script
other_sector_commit = pd.read_csv('~/Desktop/Founders_Pledge/affectable_emissions_active_repo/Affectable-Emissions/data_files/tong_country_sector_CO2.csv', index_col = 0).fillna(0) # Tong data on emissions from other sectors

# Considered Emissions
# This data is in units of Mt (million tonnes per annum, from GEM)
gas_consid = pd.read_csv('~/Desktop/Founders_Pledge/affectable_emissions_active_repo/Affectable-Emissions/data_files/gas-considered.csv', index_col = 0).fillna(0)
# This data is in units of Mt (million tonnes per annum, from GEM)
coal_consid = pd.read_csv('~/Desktop/Founders_Pledge/affectable_emissions_active_repo/Affectable-Emissions/data_files/coal-considered.csv', index_col = 0).fillna(0)
# This data was in units of Gt, and gets converted to Mt (is multiplied by 1000) in the commit_Tong-GEM.ipynb script
steel_consid = pd.read_csv('~/Desktop/Founders_Pledge/affectable_emissions_active_repo/Affectable-Emissions/data_files/steel-considered.csv', index_col = 0).fillna(0)
# This data was in units of Gt, and gets converted to Mt (is multiplied by 1000) in the commit_Tong-GEM.ipynb script
industry_consid = pd.read_csv('~/Desktop/Founders_Pledge/affectable_emissions_active_repo/Affectable-Emissions/data_files/otherindustry-considered.csv', index_col = 0).fillna(0)

Timeseries vars:
- Total coal capacity per year
- Total electricity committed per year
- Total other sectors per year

Create singular vars:
- Total considered emissions
- Total committed emissions
- Conisdered + committed + scenario

In [ ]:
# Write a function that sums variables row wise given the name of the variables to create a total
def make_custom_total(df, new_col_name):#, operation):
    var_sum = []

    for year in range(2021,2101):
        cols_to_total = []

        for col_name in df.columns[1:]:
            if 'total' not in col_name and str(year) == str(col_name[-4:]):

                    cols_to_total.append(col_name)
        
        df[new_col_name + str(year)] = np.nansum(df[cols_to_total], axis = 1)
    
    return(df)

In [ ]:
# coal.permitted + coal.pre-permit + coal.announced + coal.construction = coal.total
coal = make_custom_total(coal_consid, 'coal.total.')
# gas.commit + coal.commit = electricity.total
electricity = make_custom_total(electricity_commit, 'electricity.total.')
# sum(Other Energy, International Transport, Road Transport, Industry, Other Transport, Residential, Commerical) = sectors.total
sector_commit = make_custom_total(other_sector_commit, 'sectors.total.')

In [ ]:
A = pd.merge(coal, gas_consid, how = 'outer')
B = A.merge(industry_consid, how = 'outer')
all_consid = B.merge(steel_consid, how = 'outer')

# combin electricity and sector commit data into a committed emissions df
# all_commit is longer than either component df because each includes countries not in the other
all_commit = pd.merge(electricity, sector_commit, how = 'outer')

In [ ]:
# #list of countries which have no (or very little) considered emissions
# consid[['country','consid.total.2027']][consid['consid.total.2027'] < 1]

In [ ]:
# make total committed and considered emissions from the aggregated consid & commit dfs
# coal.total + gas.proposed + otherindustry.proposed + steel.proposed = consid.total
consid = make_custom_total(all_consid, 'consid.total.') 
# electricity.total + sectors.total = commit.total
commit = make_custom_total(all_commit, 'commit.total.')

# combine considered and committed emissions
consid_commit = pd.merge(consid, commit, how = 'outer', on = 'country')

# combine consid/commit df with scenario data, making final step in the data agg before more totals
scenario_CO2_2021_on = scenario_CO2.iloc[:,:3].join(scenario_CO2.iloc[:,174:])
consid_commit_scen = scenario_CO2_2021_on.merge(consid_commit, on = 'country', how = 'outer')

In [ ]:
# Make "expectable" emissions from scenario - (consid + commit)
def make_expectable_emits():

    for year in range(2021,2101):
        consid_commit_sum = []
        consid_commit_scen_sum = []

        for col_name in consid_commit_scen.columns[3:]:

            if str(year) == str(col_name[-4:]):

                if 'consid.total.' in col_name:
                    consid_commit_sum.append(col_name)
                    consid_commit_scen_sum.append(col_name)
                if 'commit.total.' in col_name: 
                    consid_commit_sum.append(col_name)
                    consid_commit_scen_sum.append(col_name)
                if len(col_name) == 4:
                    consid_commit_scen_sum.append(col_name)
        
        # create columns which total consid+commit and consid+commit+scen
        consid_commit_scen['commit+consid.' + str(year)] = np.nansum(consid_commit_scen[consid_commit_sum], axis = 1)
        consid_commit_scen['commit+consid+scen.' + str(year)] = np.nansum(consid_commit_scen[consid_commit_scen_sum], axis = 1)
        
        for col_name in consid_commit_scen.columns[1:]:
            if str(year) == str(col_name):
                    consid_commit_scen['expectable.' + str(year)] = consid_commit_scen[col_name] - (consid_commit_scen['commit+consid.' + str(year)])
                    
                    # This line is IMPORTANT! It makes expectable emissions values < 0 equal to 0
                    consid_commit_scen['expectable.' + str(year)][consid_commit_scen['expectable.' + str(year)] < 0] = 0

    return(consid_commit_scen)

expectable = make_expectable_emits()

In [ ]:
# merge with region mapping
regions = pd.read_csv('~/Desktop/Founders_Pledge/affectable_emissions_active_repo/Affectable-Emissions/data_files/country_mapping/countries_regions.csv').drop(columns = 'short')
# ignore messup countries not mapping at the end (for now)s
region_merge = pd.merge(expectable, regions, how = 'outer', on = 'country').iloc[:-14,:]

# put region column at the front of the df
reg = region_merge.pop('region')
region_merge.insert(1, reg.name, reg)

region_merge.to_csv('~/Desktop/Founders_Pledge/affectable_emissions_active_repo/Affectable-Emissions/data_files/master_file/affectable_emissions_MASTER.csv')